[View in Colaboratory](https://colab.research.google.com/github/whitestones011/nlp/blob/master/Word_vectors.ipynb)

In [0]:
import pandas as pd
import numpy as np
import csv

1. Download Glove pre-trained word-vectors (glove.6B.50d.txt) and save on Google Colab. The Glove txt file containes 50-dimentional vectors of 6B tokens (400K vocab, uncased).

In [0]:
# Upload files from local drive to Google Drive. 
# See more methods to upload/dowload data on https://colab.research.google.com/notebooks/io.ipynb#scrollTo=eikfzi8ZT_rW
# Remove: !rm file_name
# Rename: !mv old_file new_file

from google.colab import files
uploaded = files.upload()

2. Convert Glove txt file into pandas dataframe of shape (400000,50).

In [0]:
word_vec= pd.read_table('glove.6B.50d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [0]:
# check the shape of word vectors
word_vec.shape[0]==400000

###  Cosine similarity

Cosine similarity reflects the degree of similariy between vectors.

$$\text{Cosine_Similarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 